## Description
The objective of this notebook is to
1. Combine both train / val dataset on the dataset page to make one very big dataset
2. For classes that are NOT in the train dataset, label them as unknown 
3. Convert the images to binary file and add it to the dataframe
4. Filter for variables in test set only
5. Convert all categorical variables (for both input and output) into numerical variables
6. Save everything into a parquet

This is built on / should replace the work done on images-to-parquet.py

In [2]:

import pandas as pd


In [3]:
DF20_train_path = "../data/FungiCLEF2023_train_metadata_PRODUCTION.csv" # This is the dev training set page
DF21_val_path = "../data/FungiCLEF2023_val_metadata_PRODUCTION.csv" # This is the "validation" set on the page. We want to use this with additional data for unknown classes
public_test_path = "../data/FungiCLEF2023_public_test_metadata_PRODUCTION.csv" # Public test set
IMG_PATH = "../data/DF"

DF20_df = pd.read_csv(DF20_train_path)
DF21_df = pd.read_csv(DF21_val_path)
test_df = pd.read_csv(public_test_path)

In [25]:
train_val_df = pd.concat((DF20_df, DF21_df))

In [20]:
# These are the metadata we want to keep to train on and potentially for prediction
test_df.keys()

Index(['observationID', 'month', 'day', 'countryCode', 'locality', 'level0Gid',
       'level0Name', 'level1Gid', 'level1Name', 'level2Gid', 'level2Name',
       'Substrate', 'Latitude', 'Longitude', 'CoorUncert', 'Habitat',
       'image_path', 'filename', 'MetaSubstrate'],
      dtype='object')

In [21]:
# Additionally, we probably want to include all the phylum, genus, etc. It might be useful for additional training data.
train_val_df.keys()

Index(['observationID', 'year', 'month', 'day', 'countryCode', 'locality',
       'taxonID', 'scientificName', 'kingdom', 'phylum', 'class', 'order',
       'family', 'genus', 'specificEpithet', 'taxonRank', 'species',
       'level0Gid', 'level0Name', 'level1Gid', 'level1Name', 'level2Gid',
       'level2Name', 'ImageUniqueID', 'Substrate', 'rightsHolder', 'Latitude',
       'Longitude', 'CoorUncert', 'Habitat', 'image_path', 'class_id',
       'MetaSubstrate', 'poisonous', 'filename'],
      dtype='object')

In [41]:
cols_to_keep = list(test_df.keys()) + ['scientificName', 'kingdom', 'phylum', 'class', 'order', 'family', 'genus', 'species', 'poisonous', 'class_id']

In [42]:
train_val_df = train_val_df[cols_to_keep]
train_val_df.class_id.max()

import numpy as np
DUMMY_DATE = 361
train_val_df.loc[:, 'normalized_day'] = ((train_val_df['month'] - 1) * 30 + train_val_df['day']).fillna(DUMMY_DATE).astype(np.int16, copy=True)


In [43]:
# Make the unknown class the last class instead of -1
max_id = train_val_df.class_id.max()
train_val_df.loc[:, "class_id"] = train_val_df.class_id.apply(lambda x: max_id + 1 if x < 0 else x)

In [44]:
categoricals = ['locality', 'level0Gid', 'level1Gid', 'level2Gid', 'Substrate', 'Habitat', 'MetaSubstrate', 'kingdom', 'phylum', 'class',
       'order', 'family', 'genus', 'species']

In [13]:
# This is important to save 
mapping = {}

for col in categoricals:
    train_val_df.sort_values(by=col, ascending=True, inplace=True)
    col_numerical, col_mapping = pd.factorize(train_val_df[col], use_na_sentinel=True)
    train_val_df.loc[:, f"{col}_numerical"] = col_numerical
    mapping[col] = {v: k for k, v in enumerate(col_mapping)}


In [14]:
import pickle

CATEGORICAL_MAPPING_LOCATION = "../data/categorical_mapping.pkl"

pickle.dump(mapping, open(CATEGORICAL_MAPPING_LOCATION, 'wb'))

In [15]:
# This should be converted to a script for submission - categorical mapping for test_df

test_categoricals = ['locality', 'level0Gid', 'level1Gid', 'level2Gid', 'Substrate', 'Habitat', 'MetaSubstrate']
mapping = pickle.load(open(CATEGORICAL_MAPPING_LOCATION, 'rb'))

for col in test_categoricals:
    test_df.loc[:, col+"_numerical"] = test_df[col].apply(lambda x: mapping[col].get(x, -1))

In [10]:
train_val_df.to_csv("train_val_df.csv", index=False)
train_val_df = pd.read_csv("train_val_df.csv")

In [17]:
# This keeps on crashing + is super inefficient :( 
# Spark doesn't work on my local machine either
# Need to adapt it with images_to_parquet.py stuff

import pyarrow as pa
import pyarrow.parquet as pq
from PIL import Image
from tqdm import tqdm

IMG_DIR = "../data/DF/"

df_records = train_val_df.to_dict("records")
CHUNKS = 50
CHUNK_SIZE = len(df_records) // CHUNKS

for i in range(CHUNKS):
    records = []
    if i == CHUNKS - 1: 
        chunk = df_records[i * CHUNK_SIZE:]
    else: 
        chunk = df_records[i * CHUNK_SIZE:(i+1) *CHUNK_SIZE]
    for r in tqdm(chunk):
        img_name = r['image_path']
        if len(img_name.split("-")[0]) == 10:

            image_path = IMG_DIR + img_name.replace("JPG", "jpg")
        else: 
            image_path = IMG_DIR + img_name
        with Image.open(image_path) as im:
            r.update({
                "img_height": im.height,
                "img_widgth": im.width,
                "data": im.tobytes()
            })
        records.append(r)

    full_df = pd.DataFrame(records)

    _dataset_chunk = pa.Table.from_pandas(full_df, preserve_index=False)
    pq.write_table(_dataset_chunk, f"../data/DF_300_{i}.parquet") # TODO: To change endpoints where these parquets are stored. But we're using spark anywayz lmao

100%|██████████| 7135/7135 [00:12<00:00, 587.31it/s]

: 

In [104]:
poisonous_rare = train_val_df[train_val_df.poisonous==1].class_id.value_counts().tail(10).index
non_poisonous_rare = train_val_df[train_val_df.poisonous==0].class_id.value_counts().tail(10).index

In [107]:
rare_df = train_val_df[train_val_df.class_id.isin(poisonous_rare) | train_val_df.class_id.isin(non_poisonous_rare)]

In [108]:
### This is for the dev set only because the full thing keeps on crashing for me :(
selected_mushrooms = ['Neoboletus luridiformis (Rostk.) Gelardi, Simonini & Vizzini, 2014',
                      'Imleria badia (Fr.) Vizzini, 2014',
                      'Amanita muscaria (L.) Lam., 1783',
                      'Russula ochroleuca (Pers.) Fr.',
                      'Russula nigricans (Bull.) Fr.',
                      'Lactarius blennius (Fr.) Fr.'
                      ]

dev_set = train_val_df.scientificName.isin(selected_mushrooms)

dev_df = pd.concat((train_val_df[dev_set], train_val_df[train_val_df.class_id==1604].sample(1000), rare_df))

In [109]:
for col in categoricals:
    dev_df.sort_values(by=col, ascending=True, inplace=True)
    col_numerical, col_mapping = pd.factorize(dev_df[col], use_na_sentinel=True)
    dev_df.loc[:, f"{col}_numerical"] = col_numerical

# Need to do it for class_id too for dev set
dev_df.sort_values(by="class_id", ascending=True, inplace=True)
numerical, mapping = pd.factorize(dev_df.class_id)
dev_df.loc[:, f"class_id"] = numerical

In [112]:
# Number of cases per class
dev_df.class_id.value_counts()

class_id
26    1000
0      959
12     938
9      925
22     709
10     648
21     567
3      178
6       75
20      74
2       71
18      66
4       60
13      57
11      44
23      37
1       31
15      31
16      31
17      31
19      31
8       31
7       31
5       31
24      31
25      31
14      31
Name: count, dtype: int64

In [113]:
# This keeps on crashing + is super inefficient :( 
# Spark doesn't work on my local machine either
# Need to adapt it with images_to_parquet.py stuff

import pyarrow as pa
import pyarrow.parquet as pq
from PIL import Image
from tqdm import tqdm

IMG_DIR = "../data/DF/"

df_records = dev_df.to_dict("records")

records = []

for r in tqdm(df_records):
    img_name = r['image_path']
    if len(img_name.split("-")[0]) == 10:

        image_path = IMG_DIR + img_name.replace("JPG", "jpg")
    else: 
        image_path = IMG_DIR + img_name
    with Image.open(image_path) as im:
        r.update({
            "img_height": im.height,
            "img_widgth": im.width,
            "data": im.tobytes()
        })
    records.append(r)

full_df = pd.DataFrame(records)

  0%|          | 0/6749 [00:00<?, ?it/s]

100%|██████████| 6749/6749 [00:08<00:00, 786.11it/s]


In [114]:
full_df

,observationID,month,day,countryCode,locality,level0Gid,level0Name,level1Gid,level1Name,level2Gid,...,kingdom_numerical,phylum_numerical,class_numerical,order_numerical,family_numerical,genus_numerical,species_numerical,img_height,img_widgth,data
0,2238437186,9.0,11.0,DK,Rebæk Sø,DNK,Denmark,DNK.1_1,Hovedstaden,DNK.1.23_1,...,1,1,0,2,3,2,6,200,300,b'\x9b\x8e\x85\x9b\x8e\x85\x9c\x8f\x86\x9c\x8f...
1,2425493335,10.0,8.0,DK,Færgelunden,DNK,Denmark,DNK.1_1,Hovedstaden,DNK.1.11_1,...,1,1,0,2,3,2,6,360,300,"b',3\x127A\x1f>L(ET-HT.>D""36\x195:#HZDZ|cIz](^..."
2,2868472485,10.0,3.0,DK,Store Hareskov,DNK,Denmark,DNK.1_1,Hovedstaden,DNK.1.12_1,...,1,1,0,2,3,2,6,225,300,b'O[GbkXip`|\x80roqfwzq\xa2\xa7\xa0\x8c\x96\x8...
3,2238345070,9.0,27.0,DK,Rødding,DNK,Denmark,DNK.5_1,Syddanmark,DNK.5.21_1,...,1,1,0,2,3,2,6,300,300,b'\x15\x12\x1b\x12\x0c\x16\x12\x0c\x16\x1a\x12...
4,2427872788,10.0,17.0,SE,Hedegärde,SWE,Sweden,SWE.21_1,Västra Götaland,SWE.21.5_1,...,1,1,0,2,3,2,6,300,300,"b']qoBUS#64\x1e.-$0.\x18""!\x0b\x14\x11\x10\x16..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6744,3126949303,5.0,28.0,DK,Lemvig,DNK,Denmark,DNK.2_1,Midtjylland,DNK.2.8_1,...,1,0,7,51,78,164,266,225,300,b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00...
6745,3052833407,3.0,2.0,DK,Rugård Gods,DNK,Denmark,DNK.2_1,Midtjylland,DNK.2.18_1,...,1,0,7,51,78,164,266,240,300,b'\xe8\xfd\xec\xe9\xfe\xed\xec\xfe\xee\xed\xff...
6746,3358350449,9.0,24.0,DK,Velling Kalv (inkl Velling Gavl),DNK,Denmark,DNK.2_1,Midtjylland,DNK.2.14_1,...,1,0,11,45,101,163,265,169,300,b'\x1a\x1d\x08 #\x0e#&\x11 #\x0e\x1c\x1f\n\x19...
6747,3412520375,11.0,20.0,DK,Mågekolonien,DNK,Denmark,DNK.5_1,Syddanmark,DNK.5.7_1,...,2,4,14,50,111,161,262,200,300,b'\x11\r\x04!\x18\x0f\x16\x0c\x02\r\x00\x00\'\...


In [115]:
from sklearn.model_selection import train_test_split

train, val = train_test_split(full_df, test_size=0.2, stratify=full_df.class_id)

In [116]:
_dev_train = pa.Table.from_pandas(train, preserve_index=False)
_dev_val = pa.Table.from_pandas(val, preserve_index=False)
pq.write_table(_dev_train, "../data/dev_train.parquet")
pq.write_table(_dev_val, "../data/dev_val.parquet")